# Joint problem: Avoid 1 car and 1 pedestrian

**Load dependencies**

In [1]:
rng = MersenneTwister(2);

In [2]:
using AutomotivePOMDPs
using MDPModelChecking

In [3]:
using GridInterpolations, StaticArrays, POMDPs, POMDPToolbox, AutoViz, AutomotiveDrivingModels, Reel
using LocalApproximationValueIteration
using DiscreteValueIteration, DeepQLearning, DeepRL
using ProgressMeter, Parameters, JLD

In [4]:
cam = FitToContentCamera(0.)

AutoViz.FitToContentCamera(0.0)

In [5]:
include("masking.jl")
include("util.jl")
include("render_helpers.jl")
include("masked_dqn.jl")

## Driving environment

In [6]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =  [VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

## Discretized MDPs

In [7]:
ped_mdp = PedMDP(env = env, vel_res=2., pos_res=2., ped_type=VehicleDef(AgentClass.PEDESTRIAN, 1.0, 3.0), ped_birth=0.7)
car_mdp = CarMDP(env = env, vel_res=2., pos_res=2.);

In [8]:
threshold = 0.9999
ped_mask_file = "pedmask_new.jld"
car_mask_file = "carmask_new.jld"
ped_mask_data = load(ped_mask_file)
car_mask_data = load(car_mask_file)
ped_mask = SafetyMask(ped_mdp, StormPolicy(ped_mdp, ped_mask_data["risk_vec"], ped_mask_data["risk_mat"]), threshold)
car_mask = SafetyMask(car_mdp, StormPolicy(car_mdp, car_mask_data["risk_vec"], car_mask_data["risk_mat"]), threshold);

## Continuous space scenario

In [14]:
pomdp = UrbanPOMDP(env=env,
                   ego_goal = LaneTag(2, 1),
                   max_cars=1, 
                   max_peds=1, 
                   car_birth=0.7, 
                   ped_birth=0.3, 
                   obstacles=false, # no fixed obstacles
                   lidar=false,
                   pos_obs_noise = 0., # fully observable
                   vel_obs_noise = 0.);

In [10]:
masks = SafetyMask[ped_mask, car_mask]
ids = [101, 2]
joint_mask = JointMask([ped_mdp, car_mdp], masks, ids)
rand_pol = RandomMaskedPOMDPPolicy(joint_mask, pomdp, rng);

In [11]:
hr = HistoryRecorder(rng=rng, max_steps=100)
s0 = initialstate(pomdp, rng)
o0 = generate_o(pomdp, s0, rng)
up = FastPreviousObservationUpdater{UrbanObs}()
b0 = initialize_belief(up, o0)
@time hist2 = simulate(hr, pomdp, rand_pol, up, b0, s0);

  6.779633 seconds (7.04 M allocations: 277.721 MiB, 4.96% gc time)


In [12]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
safe_actions_hist = h.ainfo_hist
push!(action_hist, CarMDPAction(NaN))
push!(safe_actions_hist, [CarMDPAction(NaN)])
duration, fps, render_hist = animate_states(pomdp, state_hist, action_hist, safe_actions_hist, joint_mask, interp=true)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpq7lkxb", 0x0000000000000065, 2.0, nothing)

In [ ]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, rand_pol, n_ep=1000, max_steps=100, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

In [46]:
include("masking.jl")

In [29]:
sc = state_hist[68];
# bc = h.belief_hist[85]
# safe_actions(pomdp, joint_mask, bc)

In [34]:
egoid = 2
model = pomdp.models[egoid]
AutomotiveDrivingModels.observe!(model, sc, env.roadway, egoid)
AutomotiveDrivingModels.observe!(model, sc, env.roadway, egoid)
model.intersection_driver.priority
AutomotivePOMDPs.ttc_check(model.intersection_driver, sc, env.roadway, egoid)
model.intersection_driver.a

AutomotivePOMDPs.LonAccelDirection(1.0, 1)

In [25]:
model = pomdp.models[101]
AutomotiveDrivingModels.observe!(model, sc, env.roadway, egoid)
AutomotiveDrivingModels.observe!(model, sc, env.roadway, egoid)
model.a

AutomotivePOMDPs.ConstantSpeedDawdling(0.0, 0.0)

In [70]:
pomdp.models[egoid].intersection_driver.priority

false

In [13]:
egoid = 2
model = pomdp.models[egoid]
scene = state_hist[end]
roadway = env.roadway
AutomotiveDrivingModels.observe!(model.navigator, scene, roadway, egoid)
AutomotiveDrivingModels.observe!(model.intersection_driver, scene, roadway, egoid)
for driver in model.crosswalk_drivers
    AutomotiveDrivingModels.observe!(driver, scene, roadway, egoid)
end
a_lon_crosswalks = minimum([driver.a.a_lon for driver in model.crosswalk_drivers])
a_lon_crosswalks

-1.3270381184170816

In [14]:
crosswalk = model.crosswalk_drivers[2].crosswalk
conflict_lanes = model.crosswalk_drivers[2].conflict_lanes;

In [15]:
ped = scene[findfirst(scene, 101)]
AutomotivePOMDPs.is_crossing(ped, crosswalk, conflict_lanes, scene, roadway)

false

In [15]:
@showprogress for ep=1:10000
    hr = HistoryRecorder(rng=rng, max_steps=100)
    s0 = initialstate(pomdp, rng)
    o0 = generate_o(pomdp, s0, rng)
    up = FastPreviousObservationUpdater{UrbanObs}()
    b0 = initialize_belief(up, o0)
    hist2 = simulate(hr, pomdp, rand_pol, up, b0, s0)
    if sum(hist2.reward_hist .< 0.) != 0.
        println("Crash")
        break
    end
end

Progress:   0%|                                         |  ETA: 2:47:21

Crash


Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


## Visualize trained policy

In [24]:
policy = DeepQLearning.restore(problem_file="jointmdp-log/log4/problem.jld", weights_file="jointmdp-log/log4/weights.jld");

LoadError: [91mFile jointmdp-log/log4/problem.jld cannot be found[39m

In [14]:
masked_policy = MaskedDQNPolicy(pomdp, policy, joint_mask);

In [15]:
function POMDPToolbox.action_info(policy::MaskedDQNPolicy, s)
    return action(policy, s), safe_actions(policy.problem, policy.mask, s)
end

In [44]:
hr = HistoryRecorder(rng=rng, max_steps=100)
s0 = initialstate(pomdp, rng)
o0 = generate_o(pomdp, s0, rng)
up = FastPreviousObservationUpdater{UrbanObs}()
b0 = initialize_belief(up, o0)
@time hist2 = simulate(hr, pomdp, masked_policy, up, b0, s0);

  0.455220 seconds (3.55 M allocations: 113.499 MiB, 13.66% gc time)


In [45]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
safe_actions_hist = h.ainfo_hist
push!(action_hist, CarMDPAction(NaN))
push!(safe_actions_hist, [CarMDPAction(NaN)])
duration, fps, render_hist = animate_states(pomdp, state_hist, action_hist, safe_actions_hist, joint_mask, interp=false)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpy2VmWB", 0x0000000000000049, 2.0, nothing)

In [22]:
egoid = 2
model = pomdp.models[egoid]
scene = state_hist[62]
roadway = env.roadway
observe!(model, scene, roadway, 2)
AutomotiveDrivingModels.observe!(model.navigator, scene, roadway, egoid)
AutomotiveDrivingModels.observe!(model.intersection_driver, scene, roadway, egoid)
for driver in model.crosswalk_drivers
    AutomotiveDrivingModels.observe!(driver, scene, roadway, egoid)
end
a_lon_crosswalks = minimum([driver.a.a_lon for driver in model.crosswalk_drivers])
model = model.crosswalk_drivers[1]

AutomotivePOMDPs.CrosswalkDriver
  a: AutomotivePOMDPs.LonAccelDirection
  navigator: AutomotivePOMDPs.RouteFollowingIDM
  crosswalk: AutomotiveDrivingModels.Lane
  conflict_lanes: Array{AutomotiveDrivingModels.Lane}((6,))
  intersection_entrances: Array{AutomotiveDrivingModels.Lane}((3,))
  ped_model: AutomotivePOMDPs.ConstantPedestrian
  ped_start: Float64 4.0
  stop_delta: Float64 0.7
  accel_tol: Float64 0.1
  d_tol: Float64 0.5
  yield: Bool true
  priority: Bool true
  stop: Bool false
  wait_list: Array{Int64}((0,)) Int64[]
  clear: Bool false
  debug: Bool false


In [23]:
ego = scene[findfirst(scene, egoid)]
lane = get_lane(roadway, ego)
cw_length = get_end(model.crosswalk)
cw_center = get_posG(Frenet(model.crosswalk, cw_length/2), roadway)
collision_point = VecSE2(cw_center.x+model.crosswalk.width/2, ego.state.posG.y)
collision_point_posF = Frenet(proj(collision_point, lane, roadway, move_along_curves=false), roadway)
has_passed = lane ∈ model.conflict_lanes && (ego.state.posF.s > collision_point_posF.s)

false

In [29]:
@showprogress for ep=1:20
    hr = HistoryRecorder(rng=rng, max_steps=100)
    s0 = initialstate(pomdp, rng)
    o0 = generate_o(pomdp, s0, rng)
    up = FastPreviousObservationUpdater{UrbanObs}()
    b0 = initialize_belief(up, o0)
    hist2 = simulate(hr, pomdp, rand_pol, up, b0, s0)
    if n_steps(hist2) >= 100
        println("time out!")
        break
    end
end

time out!


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [33]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, masked_policy, n_ep=1000, max_steps=100, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

167.059055 seconds (1.50 G allocations: 46.722 GiB, 14.73% gc time)
Summary for 1000 episodes: 
Average reward: 0.302 
Average # of steps: 29.999 
Average # of violations: 0.000 
